In [ ]:
from .utils import compression_for_parquet

: 

In [ ]:
barnabe-sil/dengue/config

In [ ]:
""" 
# Carregar o arquivo CSV
file_path = '/Users/silmara.barnabe/Downloads/mestrado/dengue/dengue_venv/data/'
filename = 'tbEstabelecimento202503.csv'
full_file_path = os.path.join(file_path, filename)

df = pd.read_csv(full_file_path, delimiter=';', dtype=str, encoding='ISO-8859-1')

# Definir o tamanho máximo dos arquivos em MB
max_size_mb = 25  # Tamanho máximo de 25MB para cada arquivo Parquet

# Tentar diferentes compressões
compressions = ['brotli']
for compression in compressions:
    print(f"\nIniciando a compressão com {compression}...")
    save_parquet_in_chunks(df, file_path, max_size_mb, compression=compression)
"""


Iniciando a compressão com brotli...
Arquivo /Users/silmara.barnabe/Downloads/mestrado/dengue/dengue_venv/data/_part_1.parquet gerado com tamanho: 26.84 MB
Tamanho maior que 25.0MB. Ajustando a partição...
Arquivo /Users/silmara.barnabe/Downloads/mestrado/dengue/dengue_venv/data/_part_2.parquet gerado com tamanho: 27.18 MB
Tamanho maior que 25.0MB. Ajustando a partição...
Arquivo /Users/silmara.barnabe/Downloads/mestrado/dengue/dengue_venv/data/_part_3.parquet gerado com tamanho: 0.03 MB
Processo de partição concluído.



```python
# Exemplo de uso:

## Carregar o arquivo CSV
file_path = 'seu_arquivo.csv'
df = pd.read_csv(file_path, delimiter=';', dtype=str, encoding='ISO-8859-1')

## Tentar diferentes compressões
compressions = ['snappy', 'gzip', 'brotli', 'lz4']
for compression in compressions:
    print(f"\nIniciando a compressão com {compression}...")
    save_parquet_in_chunks(df, 'seu_arquivo', compression=compression)
```

In [14]:
import pandas as pd
import os

# Definir o caminho do diretório e o nome do arquivo
file_path = '/Users/silmara.barnabe/Downloads/mestrado/dengue/dengue_venv/data/'
file_name = 'tbEstabelecimento202503.csv'  # Nome correto do arquivo CSV

# Criar o caminho completo para o arquivo
full_file_path = os.path.join(file_path, file_name)

# Carregar o arquivo CSV
df = pd.read_csv(full_file_path, delimiter=';', dtype=str, encoding='ISO-8859-1')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567123 entries, 0 to 567122
Data columns (total 56 columns):
 #   Column                                       Non-Null Count   Dtype 
---  ------                                       --------------   ----- 
 0   CO_UNIDADE                                   567123 non-null  object
 1   CO_CNES                                      567123 non-null  object
 2   NU_CNPJ_MANTENEDORA                          115370 non-null  object
 3   TP_PFPJ                                      567123 non-null  object
 4   NIVEL_DEP                                    567123 non-null  object
 5   NO_RAZAO_SOCIAL                              567122 non-null  object
 6   NO_FANTASIA                                  567033 non-null  object
 7   NO_LOGRADOURO                                567122 non-null  object
 8   NU_ENDERECO                                  566808 non-null  object
 9   NO_COMPLEMENTO                               274726 non-null  object
 

In [16]:
# Tentar diferentes compressões
compressions = ['brotli']
for compression in compressions:
    print(f"\nIniciando a compressão com {compression}...")
    save_parquet_in_chunks(df, file_path, 'tbEstabelecimento202503', compression=compression)


Iniciando a compressão com brotli...
Arquivo /Users/silmara.barnabe/Downloads/mestrado/dengue/dengue_venv/data/_part_1.parquet gerado com tamanho: 26.84 MB


TypeError: '>' not supported between instances of 'float' and 'str'

In [17]:
import pandas as pd
import os
import math

# Função para salvar o DataFrame em arquivos Parquet de tamanho máximo de 25MB
def save_parquet_in_chunks(df, base_filename, max_size_mb=25, compression='snappy'):
    """
    Salva um DataFrame em múltiplos arquivos Parquet de tamanho máximo especificado (em MB).
    
    Parâmetros:
    df (DataFrame): O DataFrame que será dividido e salvo.
    base_filename (str): O nome base para os arquivos Parquet gerados.
    max_size_mb (int ou float): O tamanho máximo de cada arquivo Parquet em MB (padrão é 25MB).
    compression (str): O tipo de compressão a ser utilizado (pode ser 'snappy', 'gzip', 'brotli', 'lz4').

    Retorna:
    None
    """
    
    # Garantir que max_size_mb seja um número
    max_size_mb = float(max_size_mb)
    
    # Criar uma pasta para a compressão, caso não exista
    output_dir = os.path.join(base_filename, compression)
    os.makedirs(output_dir, exist_ok=True)  # Cria a pasta se não existir

    # Inicializando o índice para a partição
    partition_index = 0
    start_index = 0
    chunk_size = len(df)  # Tamanho do DataFrame inteiro
    total_size = df.memory_usage(deep=True).sum() / (1024 * 1024)  # Tamanho em MB

    # Salvar as divisões como arquivos Parquet
    while start_index < chunk_size:
        partition_index += 1
        # Divida o DataFrame em partes com base no número de linhas
        end_index = start_index + (chunk_size // 2)  # Inicialmente dividir em metades
        
        # Verificar o tamanho do arquivo Parquet
        chunk_df = df.iloc[start_index:end_index]  # Subconjunto do DataFrame
        
        chunk_filename = os.path.join(output_dir, f"{base_filename}_part_{partition_index}.parquet")
        
        # Salvar em Parquet com a compressão desejada
        chunk_df.to_parquet(chunk_filename, compression=compression)
        
        # Verificar o tamanho do arquivo gerado
        file_size = os.path.getsize(chunk_filename) / (1024 * 1024)  # Tamanho em MB
        print(f"Arquivo {chunk_filename} gerado com tamanho: {file_size:.2f} MB")
        
        # Ajustar a divisão caso o arquivo gerado seja maior que 25MB
        if file_size > max_size_mb:
            print(f"Tamanho maior que {max_size_mb}MB. Ajustando a partição...")
            chunk_df = df.iloc[start_index:end_index - (chunk_size // 4)]  # Dividir ainda mais
            chunk_df.to_parquet(chunk_filename, compression=compression)

        # Atualizar o índice para a próxima parte
        start_index = end_index

    print("Processo de partição concluído.")

# Carregar o arquivo CSV
file_path = '/Users/silmara.barnabe/Downloads/mestrado/dengue/dengue_venv/data/'
filename = 'tbEstabelecimento202503.csv'
full_file_path = os.path.join(file_path, filename)

df = pd.read_csv(full_file_path, delimiter=';', dtype=str, encoding='ISO-8859-1')

# Tentar diferentes compressões
compressions = ['brotli']
for compression in compressions:
    print(f"\nIniciando a compressão com {compression}...")
    save_parquet_in_chunks(df, file_path, 'tbEstabelecimento202503', compression=compression)


KeyboardInterrupt: 